# InfoFactAgents User Interface
This Jupyter Notebook implements a web user interface to InfoFactAgents

In [1]:
!pip install gradio requests --quiet

In [ ]:
import gradio as gr
import requests
import os
from urllib.parse import urlparse

# JavaScript function to enforce the dark theme
js_func = """
function refresh() {
    const url = new URL(window.location);

    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

# Read authentication credentials from environment variables
USERNAME = os.environ.get("INFOFACT_USERNAME")
PASSWORD = os.environ.get("INFOFACT_PASSWORD")

def authenticate(username, password):
    """Simple authentication function."""
    return username == USERNAME and password == PASSWORD

def logout(request: gr.Request):
    """Logout handler."""
    print("User logged out.")

def append_debug_message(debug_messages, message):
    """Appends a timestamped debug message to the debug log."""
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    debug_messages.append(f"{timestamp} D {message}")
    return debug_messages

def fetch_agents(backend_url, session_id, debug_messages):
    """Fetches the list of agents from the backend and logs the communication."""
    try:
        debug_messages = append_debug_message(debug_messages, f"Sending request to fetch agents: {backend_url}")
        response = requests.post(backend_url, data={"Body": "list_agents", "SessionID": session_id})
        if response.status_code == 200:
            response_data = response.json()
            session_id = response_data.get("SessionID", session_id)  # Update session ID
            agents_text = response_data.get("Response", "")
            agent_descriptions = [line for line in agents_text.split("\n") if " - " in line]
            debug_messages = append_debug_message(debug_messages, f"Received agents: {agent_descriptions}")
            return agent_descriptions, session_id, debug_messages
        else:
            error_message = f"Error: Unable to fetch agents. Status code: {response.status_code}"
            debug_messages = append_debug_message(debug_messages, error_message)
            return [error_message], session_id, debug_messages
    except requests.RequestException as e:
        error_message = f"Request failed: {e}"
        debug_messages = append_debug_message(debug_messages, error_message)
        return [error_message], session_id, debug_messages

def analyze(prompt, backend_url, selected_agents, session_id, debug_messages):
    """Sends the article or URL to the backend for analysis and logs the communication."""
    try:
        from urllib.parse import urlparse
        parsed_url = urlparse(prompt)
        if parsed_url.scheme in ["http", "https"]:
            debug_messages = append_debug_message(debug_messages, f"Fetching article content from URL: {prompt}")
            try:
                response = requests.get(prompt)
                response.raise_for_status()
                prompt = response.text
                debug_messages = append_debug_message(debug_messages, "Successfully fetched article content.")
            except requests.RequestException as e:
                error_message = f"Error fetching article content: {e}"
                debug_messages = append_debug_message(debug_messages, error_message)
                return error_message, session_id, debug_messages

        selected_agents = [agent.split(" - ")[0] for agent in selected_agents if agent]
        debug_messages = append_debug_message(debug_messages, f"Sending analysis request with agents: {selected_agents}")
        response = requests.post(
            backend_url,
            data={"Body": prompt, "Agents": ",".join(selected_agents), "SessionID": session_id},
        )
        if response.status_code == 200:
            response_data = response.json()
            session_id = response_data.get("SessionID", session_id)
            response_text = response_data.get("Response", "")
            debug_messages = append_debug_message(debug_messages, f"Received analysis response: {response_text}")
            return response_text, session_id, debug_messages
        else:
            error_message = f"Error: {response.status_code} - {response.text}"
            debug_messages = append_debug_message(debug_messages, error_message)
            return error_message, session_id, debug_messages
    except requests.RequestException as e:
        error_message = f"Request failed: {e}"
        debug_messages = append_debug_message(debug_messages, error_message)
        return error_message, session_id, debug_messages

with gr.Blocks(js=js_func) as interface:
    debug_messages = gr.State([])  # State to store debug messages
    session_id = gr.State("")  # State to store the SessionID

    with gr.Tab("Article"):

        # Display flow.jpg and a text box with project information side by side
        with gr.Row(equal_height=True):
            with gr.Column(scale=2):
                gr.Textbox(value="Welcome to the InfoFactAgents Fact-Checking Tool!\n\nThis tool is designed to analyze the trustworthiness of news articles and other textual content. It leverages advanced AI agents and external tools for factual consistency, metadata analysis, and sentiment evaluation.\n\nThis exploratory tool was a byproduct of Medialukutaito Infotester -coursework in Tampere University of Applied Sciences.\n\nProject Team:\n- Essi Salonen\n- Mika Wilen\n- Joona Sjöholm\n- Jasperi Järveläinen\n\nVisit the GitHub repository at https://github.com/mwtuni/infofactagents to explore the codebase and contribute!", label="Project Info", interactive=False)
            with gr.Column(scale=1):
                gr.Image("flow.jpg", label="Workflow Overview", elem_id="workflow-image")

        # Add custom CSS
        gr.HTML("""
        <style>
            #workflow-image img {
                height: 100%;
                object-fit: contain;
            }
        </style>
        """)

        # Backend Section
        with gr.Row():
            backend_url = gr.Textbox(label="Backend URL", value="http://localhost:5000/infofactagents", interactive=True)

        # Agents Section
        with gr.Row():
            enabled_agents = gr.CheckboxGroup(label="Enable Agents", choices=[], interactive=True)

        # Article Section
        with gr.Row():
            prompt = gr.Textbox(label="Enter news article", lines=5, interactive=True)

        # Analyze Button and Trustworthiness Output
        with gr.Row():
            send_button = gr.Button("Analyze")
        with gr.Row():
            output = gr.Textbox(label="Trustworthiness Analysis", lines=10, interactive=False)

        def update_agents(backend_url, session_id, debug_messages):
            agents, session_id, updated_debug = fetch_agents(backend_url, session_id, debug_messages)
            return gr.update(choices=agents), session_id, updated_debug

        # Automatically fetch agents when the interface starts
        interface.load(
            fn=update_agents,
            inputs=[backend_url, session_id, debug_messages],
            outputs=[enabled_agents, session_id, debug_messages],
        )

        send_button.click(
            fn=analyze,
            inputs=[prompt, backend_url, enabled_agents, session_id, debug_messages],
            outputs=[output, session_id, debug_messages],
        )

    with gr.Tab("Debug") as debug_tab:
        debug_output = gr.Textbox(label="Debug Info", lines=15, interactive=False)

        # Refresh debug info whenever the Debug tab is activated
        debug_tab.select(
            fn=lambda debug_messages: "\n".join(debug_messages),
            inputs=debug_messages,
            outputs=debug_output,
        )

    # Logout Button
    logout_button = gr.Button(value="Logout")
    logout_button.click(logout)

    # Automatically refresh the page to enforce the dark theme
    gr.HTML("<script>refresh();</script>")

interface.launch(auth=authenticate, share=False, server_port=5042, server_name="0.0.0.0")

IndentationError: expected an indented block after 'with' statement on line 100 (2767238709.py, line 103)

In [ ]:
# Display flow.jpg and a text box with project information side by side
with gr.Row(equal_height=True):
    with gr.Column(scale=2):
        gr.Textbox(value="Welcome to the InfoFactAgents Fact-Checking Tool!\n\nThis tool is designed to analyze the trustworthiness of news articles and other textual content. It leverages advanced AI agents and external tools for factual consistency, metadata analysis, and sentiment evaluation.\n\nThis exploratory tool was a byproduct of Medialukutaito Infotester -coursework in Tampere University of Applied Sciences.\n\nProject Team:\n- Essi Salonen\n- Mika Wilen\n- Joona Sjöholm\n- Jasperi Järveläinen\n\nVisit the GitHub repository at https://github.com/mwtuni/infofactagents to explore the codebase and contribute!", label="Project Info", interactive=False)
    with gr.Column(scale=1):
        gr.Image("flow.jpg", label="Workflow Overview", scale=0.3, style={"height": "100%"})

In [ ]:
# Display flow.jpg and a text box with project information side by side
with gr.Row(equal_height=True):
    with gr.Column(scale=2):
        gr.Textbox(value="Welcome to the InfoFactAgents Fact-Checking Tool!\n\nThis tool is designed to analyze the trustworthiness of news articles and other textual content. It leverages advanced AI agents and external tools for factual consistency, metadata analysis, and sentiment evaluation.\n\nThis tool was a byproduct of Medialukutaito Infotester -coursework in Tampere University of Applied Sciences.\n\nProject Team:\n- Essi Salonen\n- Mika Wilen\n- Joona Sjöholm\n- Jasperi Järveläinen\n\nWe welcome contributions to improve and expand this tool. Visit the [GitHub repository](https://github.com/mwtuni/infofactagents) to explore the codebase and contribute!", label="Project Info", interactive=False)
    with gr.Column(scale=1):
        gr.Image("flow.jpg", label="Workflow Overview", scale=0.3)